# Python-Chess

## 1. Introduction
python-chess is a pure‐Python library for working with chess data: board representation, move generation, PGN/Polyglot I/O, and UCI engine communication. It underpins SafeChess’s core logic, giving us:

Accurate FEN/PGN parsing and serialization
Full move legality checks, castling, en passant
Easy integration with UCI engines (Stockfish)
## 2. Paradigm & Key Objects
We use python-chess in three main contexts:


|Context|Core Objects|
|---|---|
|Position & Moves|`Board`, `Move`, `SquareSet`|
|Game Streams|`Game`, `GameNode`, `chess.pgn`|
|Engine I/O|`SimpleEngine`, `uci module`|

`Board`
Holds the full position (pieces, turn, castling rights, moves stack).
`Move`
Represents a single move, in UCI or SAN form.
`Game` / `GameNode`
A tree structure for PGN games; each `GameNode` has a `board()` and `mainline_moves()`.
`SimpleEngine`
A thin wrapper around UCI engines, giving `.analyse()` and `.play()` methods.

## 3. Code examples

### Installation 

to install `python-chess`, we do

```bash
pip install python-chess
```

### Basic Moves

In [1]:
import chess

In [6]:
#create a chess baord with pieces in starting positions
board = chess.Board()
#display the board
print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


we have several formats for moves:

**Universal Chess Interface (UCI)** aka Coordinate Notation - 

"`e2e4`" (piece moves from `e2` to `e4` )

A nice and strict "from square to square" notation, good for engines without the need for handling edge cases around standard algebraic notation for ambiguous moves like `Rab1` where either rook could move to `b1`.

**Standard Algebraic Notation (SAN)** -  

"`e4`" (pawn moves to e4 (from `e3` or `e2`))

The normal form seen in literature - more human readable. format can vary, following this pattern:

Piece identifier, additional file or rank (in that order of preference) value if ambiguous, piece taken?, final coordinate, check or checkmate flag.

so if we have two Rooks on the 1st rank, one on `a` and one on `f`, and an opposing rook on `e1`, and the rook on `a1` takes the opposing rook, resulting in a checkmate for White:
`R` (Rook identifier) `a` (on a, as either rook could have made this move), `x` (takes the piece on the final square) `e1` (moves to `e1`), `#` (checkmating the black King) - or `Raxe1#`

the pieces are: pawn (omitted), Knight (N), Bishop (B), Rook (R), Queen (Q) and King (K). 

When a pawn makes a capture, its original **rank** is included too. So a black pawn on `d4` taking a white pawn on `e3` will look like [`e3, dxe3`].

Remember the edge case around en passant, where the following exchange: [`e4, dxe3`] is legal, and is the only condition we take a piece without landing on its square.

Another unusual notation is `0-0` or `O-O` for Kingside castling, and `0-0-0` or `O-O-O` for Queenside castling.

Finally, promotion is noted as: final coordinate, =, new piece value. so a Queen promotion on `e8` is `e8=Q`, and underpromotion to a knight on c8 is `c8=N`.


**We will handle UCI in our engine, and display SAN for human readability. We can also use unicode characters or emojis for the pieces too**



In [ ]:
move = chess.Move.from_uci("e2e4")
#check if the move is legal
if move in board.legal_moves:
    #make the move
    board.push(move)
    print(board)
else:
    print("Illegal move")

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


In [8]:
# converting the next move to SAN
move_san = board.san(chess.Move.from_uci("e7e5"))
print(f"Move in SAN: {move_san}")

Move in SAN: e5


### Inpecting all legal moves

In [15]:
print(list(board.legal_moves))

#Filter board by piece
king_squares = [square for square in board.pieces(chess.KING, chess.WHITE)]
print("white king is on the coordinates: ", king_squares)

[Move.from_uci('g8h6'), Move.from_uci('g8f6'), Move.from_uci('b8c6'), Move.from_uci('b8a6'), Move.from_uci('h7h6'), Move.from_uci('g7g6'), Move.from_uci('f7f6'), Move.from_uci('e7e6'), Move.from_uci('d7d6'), Move.from_uci('c7c6'), Move.from_uci('b7b6'), Move.from_uci('a7a6'), Move.from_uci('h7h5'), Move.from_uci('g7g5'), Move.from_uci('f7f5'), Move.from_uci('e7e5'), Move.from_uci('d7d5'), Move.from_uci('c7c5'), Move.from_uci('b7b5'), Move.from_uci('a7a5')]
white king is on the coordinates:  [4]


### FEN and PGN I/O

#### FEN 
FEN (Forsyth-Edwards Notation) is the way of storing a board at a given state and position at one point in "time".

e.g. `rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1`

which means, **going backwards from a8 to h1**, with lowercase letters for black, and Uppercase letters for white:

Rank 8: Rook, Knight, Bishop, Queen, King, Bishop, Knight, Rook

Rank 7: 8 Pawns

Rank 6: 8 Blank spaces

Rank 5: 8 Blank spaces

Rank 4: 4 Blank spaces, a WHITE Pawn (uppercase!), and then 3 blank spaces

Rank 3: 8 Blank spaces

Rank 2: 4 Pawns, a gap of 1, and 3 Pawns

Rank 1: Rook, Knight, Bishop, Queen, King, Bishop, Knight, Rook


After that is the state metadata:

**Active colour**: `b` or `w` - here Black is to move next

**Castling availability**: `K`, `Q`, `k`, `q` - here, both WHITE and black can still castle `K`ingside and `Q`ueenside

**En passant target square**: coordinate or `-` - here, as white double advanced their `e` pawn as `e2e4` or just `e4`, Black might take on `e3` if possible. If white opened with `e3` instead, a `-` would be seen as no en passant target square is available.

**Half-move clock**: How man half moves have occured since the last capture or pawn advance  - used for the 50 move draw timeout rule.

**Full moves**: How many full moves (White and Black move once per full move) in the game so far.

#### PGN
Portable Game Notation - for electronic records of games with metadata:

```pgn
[Event "F/S Return Match"]
[Site "Belgrade, Serbia JUG"]
[Date "1992.11.04"]
[Round "29"]
[White "Fischer, Robert J."]
[Black "Spassky, Boris V."]
[Result "1/2-1/2"]

1.e4 e5 2.Nf3 Nc6 3.Bb5 {This opening is called the Ruy Lopez.} 3...a6
4.Ba4 Nf6 5.O-O Be7 6.Re1 b5 7.Bb3 d6 8.c3 O-O 9.h3 Nb8 10.d4 Nbd7
11.c4 c6 12.cxb5 axb5 13.Nc3 Bb7 14.Bg5 b4 15.Nb1 h6 16.Bh4 c5 17.dxe5
Nxe4 18.Bxe7 Qxe7 19.exd6 Qf6 20.Nbd2 Nxd6 21.Nc4 Nxc4 22.Bxc4 Nb6
23.Ne5 Rae8 24.Bxf7+ Rxf7 25.Nxf7 Rxe1+ 26.Qxe1 Kxf7 27.Qe3 Qg5 28.Qxg5
hxg5 29.b3 Ke6 30.a3 Kd6 31.axb4 cxb4 32.Ra5 Nd5 33.f3 Bc8 34.Kf2 Bf5
35.Ra7 g6 36.Ra6+ Kc5 37.Ke1 Nf4 38.g3 Nxh3 39.Kd2 Kb5 40.Rd6 Kc5 41.Ra6
Nf2 42.g4 Bd3 43.Re6 1/2-1/2
```

Those seven metadata tages MUST appear in any PGN.

Note the comment.

Note that we can add Numeric Annotation Graphs or NAGs to each move, to denote assessment of move, position or commentary. There are too many to list here, but we will need to use them in the future when reporting lines back to a user.

In [20]:
fen = board.fen()
print("FEN: ", fen)

FEN:  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1


In [21]:
board2 = chess.Board(fen)
print(board2)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


In [26]:
board = chess.Board()

# PGN Reading
import chess.pgn
with open("../data/example_pgn_1.pgn") as f:
    game = chess.pgn.read_game(f)
    print(game.headers["Event"])
    for move in game.mainline_moves():
        board.push(move)

# PGN Writing
game = chess.pgn.Game()
game.headers["Event"] = "My Spike"
node = game
for uci in ["e2e4","e7e5","g1f3"]:
    move = chess.Move.from_uci(uci)
    node = node.add_variation(move)
print(game)  # full PGN

Let\\'s Play!
[Event "My Spike"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "*"]

1. e4 e5 2. Nf3 *


### Engines

We can wrap python-chess around almost any common engine with the `UCI` or `SimpleEngine` interfaces

In [27]:
import os
import chess.engine

# Launch engine
engine = chess.engine.SimpleEngine.popen_uci(os.getenv("STOCKFISH_PATH"))

# Single‐position analysis
info = engine.analyse(board, chess.engine.Limit(depth=12), multipv=3)
for i, entry in enumerate(info, start=1):
    score = entry["score"].white().score()
    pv = board.variation_san(entry["pv"])
    print(f"#{i}: {score:+} → {pv}")

engine.quit()


#1: -611 → 18. Bxd5+ Rxd5 19. Bxb6 cxb6 20. Nf3 Bxh1 21. Nbd2 Bg2 22. Rg1 Bxf3+ 23. Nxf3 Rc5 24. Rg4 Rc7
#2: -620 → 18. Nh3 Bxh1 19. f3 Bg2 20. Bxd5+ Rxd5
#3: -654 → 18. f3 Bxh1 19. Nh3 Bg2 20. a4 Kxa7
